## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config_vacation import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,-33.6971,-53.4616,69.3,88,100,3.67,overcast clouds
1,1,Hobart,AU,-42.8794,147.3294,61.0,63,40,14.97,scattered clouds
2,2,Roma,US,43.2128,-75.4557,48.2,57,1,6.91,clear sky
3,3,Barrow,US,71.2906,-156.7887,-11.2,77,1,5.75,clear sky
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,80.6,69,0,10.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,80.60,69,0,10.36,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,79.79,68,1,18.05,clear sky
12,12,Kavieng,PG,-2.5744,150.7967,84.00,72,100,15.39,overcast clouds
18,18,Kapaa,US,22.0752,-159.3190,78.80,69,20,14.97,few clouds
19,19,Vao,NC,-22.6667,167.4833,77.95,79,12,13.06,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                228
City                   228
Country                228
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,80.60,69,0,10.36,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,79.79,68,1,18.05,clear sky
12,12,Kavieng,PG,-2.5744,150.7967,84.00,72,100,15.39,overcast clouds
18,18,Kapaa,US,22.0752,-159.3190,78.80,69,20,14.97,few clouds
19,19,Vao,NC,-22.6667,167.4833,77.95,79,12,13.06,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Arraial Do Cabo,BR,80.60,clear sky,-22.9661,-42.0278,
7,Rikitea,PF,79.79,clear sky,-23.1203,-134.9692,
12,Kavieng,PG,84.00,overcast clouds,-2.5744,150.7967,
18,Kapaa,US,78.80,few clouds,22.0752,-159.3190,
19,Vao,NC,77.95,few clouds,-22.6667,167.4833,
20,Hithadhoo,MV,83.37,overcast clouds,-0.6000,73.0833,
23,Buala,SB,83.79,moderate rain,-8.1450,159.5921,
25,Ugoofaaru,MV,82.17,overcast clouds,5.6667,73.0000,
27,Poum,NC,81.16,clear sky,-20.2333,164.0167,
32,Palauig,PH,81.09,overcast clouds,15.4350,119.9045,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
hotel_df.count()

City                   228
Country                228
Max Temp               228
Current Description    228
Lat                    228
Lng                    228
Hotel Name             228
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Arraial Do Cabo,BR,80.60,clear sky,-22.9661,-42.0278,Pousada Porto Praia
7,Rikitea,PF,79.79,clear sky,-23.1203,-134.9692,Pension Maro'i
12,Kavieng,PG,84.00,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
18,Kapaa,US,78.80,few clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
19,Vao,NC,77.95,few clouds,-22.6667,167.4833,Hotel Kou-Bugny


In [11]:
clean_hotel_df.count()

City                   215
Country                215
Max Temp               215
Current Description    215
Lat                    215
Lng                    215
Hotel Name             215
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))